# Sandoz - List
* 2017: OCR
* 2018: PDF
* 2019: PDF, but HCO without total

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
#import consts

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("payment-disclosure-sandoz-switzerland-2019-de.pdf", pages='all', lattice=True)

## Format Table

In [3]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Remove by String
df_export = df_export[~df_export['name'].str.contains('Zahl der Empfänger unter', na=False)]
df_export = df_export[~df_export['name'].str.contains('Diesen Empfängern', na=False)]
df_export = df_export[~df_export['name'].str.contains('% der aggregiert', na=False)]

#Drop Rows
df_export = df_export.dropna(subset=['donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')
df_export = df_export.dropna(subset=['location'], how='all')

#Drop Column
df_export = df_export.drop(columns=['empty1'])

#Set Type
df_export = set_type_by_alphabetical_order(df_export)

#Remove \r
df_export = remove_carination(df_export, " ")

#shift Surname/Lastname
df_export = revert_name(df_export, ', ')

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Sum HCO
def f(v):
    if np.isnan(v):
        return 0
    return v
def sum_row(row):
    return (f(row['donations_grants']) + f(row['sponsorship']) + f(row['registration_fees']) + f(row['travel_accommodation'])
           + f(row['fees']) + f(row['related_expenses']))
df_export.loc[df_export.type == 'hco', 'total'] = df_export.apply(sum_row , axis=1)


#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'sandoz')

saved


In [4]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)